## Final

In [20]:
# Dependencies
import json
import requests as req
import pandas as pd
import os
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from census import Census
from states import states, states_hash, states_loc
from passwords import census_key, google_key, plotly_user, plotly_key
from NCMEC_parser import NCMEC_parser

### Pulling Data from NCMEC

In [21]:
# NCMEC website parser by state - NCMEC_parser.py
# for state in states:
#     NCMEC_parser(state)

### Combining and Cleaning NCMEC Data

In [22]:
# # combining data by state in to one df
# df = pd.DataFrame(columns = ['name','age','gender','ethnicity','NCMEC_case#','case_type','birth_year','missing_year','missing_city','missing_county','missing_state'])

# index = 0
# for state in states:
#     state_df = pd.read_csv('Resources/NCMEC_{}.csv'.format(state))
#     for i in range(len(state_df)):
#         df.set_value(index,'name',state_df['name'][i])
#         df.set_value(index,'age',state_df['age'][i])
#         df.set_value(index,'gender',state_df['gender'][i])
#         df.set_value(index,'ethnicity',state_df['ethnicity'][i])
#         df.set_value(index,'NCMEC_case#',state_df['NCMEC_case#'][i])
#         df.set_value(index,'case_type',state_df['case_type'][i])
#         df.set_value(index,'birth_year',state_df['birth_year'][i])
#         df.set_value(index,'missing_year',state_df['missing_year'][i])
#         df.set_value(index,'missing_city',state_df['missing_city'][i])
#         df.set_value(index,'missing_county',state_df['missing_county'][i])
#         df.set_value(index,'missing_state',states[state])
#         index += 1
# df

In [23]:
# # cleaning combined NCMEC data for 50 states
# df['age'] = df['age'].replace(-1,float('NaN'))
# df['gender'] = df['gender'].replace('unknown',float('NaN'))
# df['ethnicity'] = df['ethnicity'].replace('unknown',float('NaN'))
# df['ethnicity'] = df['ethnicity'].replace('amInd','native american')
# df['ethnicity'] = df['ethnicity'].replace(['blackHisp','whiteHisp'],'hispanic')
# df['case_type'] = df['case_type'].replace(['unidentified','unknown'],float('NaN'))
# df['case_type'] = df['case_type'].replace('endangeredRunaway','runaway')
# df['case_type'] = df['case_type'].replace(['Section 5779','endangeredMissing','lostInjuredMissing'],'missing')
# df['case_type'] = df['case_type'].replace(['familyAbduct','hagueCase','nonFamilyAbduct'],'abduct')
# df.head()

In [24]:
# finding latitude and longitude for missing city,state
# for i in range(len(df)):
#     try:
#         gkey = google_key
#         target_city = "{}, {}".format(df['missing_city'][i],df['missing_state'][i])
#         target_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(target_city, gkey)
#         geo_data = req.get(target_url).json()
#         lat = geo_data["results"][0]["geometry"]["location"]["lat"]
#         lng = geo_data["results"][0]["geometry"]["location"]["lng"]
#         print("location: {} lat: {} lng: {}".format(target_city,lat,lng))
#     except:
#         print("{} fail".format(i))
#         break
         


In [25]:
#df.to_csv("Resources/_NCMEC_final.csv", encoding="utf-8", index=False)

### Missing Peoples Ratio - # of Missing People vs State Population

In [26]:
# Census API Key
c = Census(census_key)

# Read Main CSV
df_NCMEC = pd.read_csv("Resources/_NCMEC_final.csv")
df_NCMEC.head()

,name,age,gender,ethnicity,NCMEC_case#,case_type,birth_year,missing_year,missing_city,missing_county,missing_state
0,DAVID BORER,36.0,male,white,722859,abduct,1981.0,1989.0,WILLOW,Matanuska Susitna,Alaska
1,MEGAN EMERICK,61.0,female,white,1080580,missing,1956.0,1973.0,SEWARD,Kenai Peninsula,Alaska
2,STELLA EVON,38.0,female,native american,919705,missing,1979.0,1996.0,BETHEL,Bethel,Alaska
3,AMY FANDEL,47.0,female,white,601234,abduct,1970.0,1978.0,STERLING,Kenai Peninsula,Alaska
4,AMY FANDEL,47.0,female,white,601234,abduct,1970.0,1978.0,STERLING,Kenai Peninsula,Alaska


In [27]:
c = Census("5e99f22848e97d5bcc53f03a358b973493473cd9")
cKey = "5e99f22848e97d5bcc53f03a358b973493473cd9"

census_data1 = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})


url1 = "https://api.census.gov/data/2014/pep/natstprc?get=STNAME,POP&for=state:*&DATE=7"
reqUrl1 = url1 + "&key="+ cKey
getData1 = req.get(reqUrl1).json()

In [28]:
# State level dataframe
df_states = pd.DataFrame(getData1,columns=getData1[0])
df_states = df_states.drop(df_states.index[0])
df_states['POP'] =  list(map(int,df_states['POP']))
df_states = df_states.drop('DATE', 1)
df_states = df_states[df_states.STNAME != 'District of Columbia']
df_states = df_states[df_states.STNAME != 'Puerto Rico Commonwealth']
df_states = df_states.reset_index(drop = True)
df_states['state_key'] = [states_hash[state] for state in list(df_states['STNAME'])]
df_states['Miss_Pop'] = list(df_NCMEC.groupby('missing_state').count()['name'])
# Devided population by 1 million for a better ratio between missing peoples and population
df_states['Miss%'] = round(df_states['Miss_Pop']/df_states['POP'] * 1000000,2)

df_states

,STNAME,POP,state,state_key,Miss_Pop,Miss%
0,Alabama,4849377,01,AL,35,7.22
1,Alaska,736732,02,AK,12,16.29
2,Arizona,6731484,04,AZ,104,15.45
3,Arkansas,2966369,05,AR,37,12.47
4,California,38802500,06,CA,557,14.35
5,Colorado,5355866,08,CO,86,16.06
6,Connecticut,3596677,09,CT,24,6.67
7,Delaware,935614,10,DE,9,9.62
8,Florida,19893297,12,FL,327,16.44
9,Georgia,10097343,13,GA,87,8.62


In [29]:
url2 = "https://api.census.gov/data/2015/acs1?get=NAME,B19013_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E&for=state:*"
reqUrl2 = url2 + "&key="+ cKey
getData2 = req.get(reqUrl2).json()

In [30]:
df_temp = pd.DataFrame(getData2, columns=getData2[0])
df_temp = df_temp.drop(df_temp.index[0])
df_temp = df_temp.drop('state', 1)
# getIncData["Total Ethnic"] =  int (getIncData["B02001_002E"]) + int (getIncData["B02001_003E"]) +int (getIncData["B02001_004E"]) + int (getIncData["B02001_005E"]) + int (getIncData["B02001_006E"]) + int (getIncData["B02001_007E"]) + int (getIncData["B02001_008E"]) + int (getIncData["B02001_009E"]) + int (getIncData["B02001_010E"])
df_temp = df_temp.rename(columns={"B19013_001E": "Household Income", 
                                      "B02001_002E": "Whites", 
                                      "B02001_003E": "African Americans",
                                      "B02001_004E": "American Indians",
                                      "B02001_005E": "Asians",
                                      "B02001_006E": "Pacific Islanders",
                                      "B02001_007E": "Others",
                                      "B02001_008E": "Two or More Races",
                                      "B02001_009E": "Two or More Inc Others",
                                      "B02001_010E": "Two or More Excl Others",
                                      "NAME": "STNAME" })

df_temp = df_temp[df_temp.STNAME != 'District of Columbia']
df_temp = df_temp[df_temp.STNAME != 'Puerto Rico Commonwealth']
df_temp = df_temp[df_temp.STNAME != 'Puerto Rico']
df_temp = df_temp.reset_index(drop = True)
df_temp

,STNAME,Household Income,Whites,African Americans,American Indians,Asians,Pacific Islanders,Others,Two or More Races,Two or More Inc Others,Two or More Excl Others
0,Alabama,44765,3329913,1303298,20503,59393,4448,54105,87319,6354,80965
1,Alaska,73355,481314,26176,101133,47295,6762,12467,63285,3171,60114
2,Arizona,51492,5285994,300685,308469,217556,11284,481631,222446,45251,177195
3,Arkansas,41995,2308047,469555,20797,42404,7639,65542,64220,6149,58071
4,California,64500,23824254,2277229,282777,5548936,157554,5300297,1753771,375682,1378089
5,Colorado,63909,4607390,224809,49138,162880,9136,209890,193331,30981,162350
6,Connecticut,71346,2747943,380571,8893,156249,519,180107,116604,28025,88579
7,Delaware,61255,654904,203863,3121,36114,334,19157,28441,3500,24941
8,Florida,49426,15357374,3280778,47032,544068,11898,528266,501856,87649,414207
9,Georgia,51244,6077438,3197929,28491,388457,4337,284084,234124,36148,197976


In [31]:
df_states = df_states.merge(df_temp, on='STNAME')

In [32]:
# Clean data and add ethnic percentages
df_states['Whites'] = list(map(int,df_states['Whites']))
df_states['Whites%'] = round(df_states['Whites']/df_states['POP']*100, 2)
df_states['African Americans'] = list(map(int,df_states['African Americans']))
df_states['African Americans%'] = round(df_states['African Americans']/df_states['POP']*100, 2)
df_states['American Indians'] = list(map(int,df_states['American Indians']))
df_states['American Indians%'] = round(df_states['American Indians']/df_states['POP']*100, 2)
df_states['Asians'] = list(map(int,df_states['Asians']))
df_states['Asians%'] = round(df_states['Asians']/df_states['POP']*100, 2)
df_states['Pacific Islanders'] = list(map(int,df_states['Pacific Islanders']))
df_states['Pacific Islanders%'] = round(df_states['Pacific Islanders']/df_states['POP']*100, 2)
df_states['Others'] = list(map(int,df_states['Others']))
df_states['Others%'] = round(df_states['Others']/df_states['POP']*100, 2)
df_states.rename(columns={'Two or More Races': 'BiRacial','Two or More Inc Others': 'BiRacial Incl Others','Two or More Excl Others':'BiRacial Excl Others'}, inplace=True)
df_states['BiRacial'] = list(map(int,df_states['BiRacial']))
df_states['BiRacial%'] = round(df_states['BiRacial']/df_states['POP']*100, 2)
df_states['BiRacial Incl Others'] = list(map(int,df_states['BiRacial Incl Others']))
df_states['BiRacial Excl Others'] = list(map(int,df_states['BiRacial Excl Others']))

'''
Gender + Education level Data if possible
'''

df_states

,STNAME,POP,state,state_key,Miss_Pop,Miss%,Household Income,Whites,African Americans,American Indians,...,BiRacial,BiRacial Incl Others,BiRacial Excl Others,Whites%,African Americans%,American Indians%,Asians%,Pacific Islanders%,Others%,BiRacial%
0,Alabama,4849377,01,AL,35,7.22,44765,3329913,1303298,20503,...,87319,6354,80965,68.67,26.88,0.42,1.22,0.09,1.12,1.80
1,Alaska,736732,02,AK,12,16.29,73355,481314,26176,101133,...,63285,3171,60114,65.33,3.55,13.73,6.42,0.92,1.69,8.59
2,Arizona,6731484,04,AZ,104,15.45,51492,5285994,300685,308469,...,222446,45251,177195,78.53,4.47,4.58,3.23,0.17,7.15,3.30
3,Arkansas,2966369,05,AR,37,12.47,41995,2308047,469555,20797,...,64220,6149,58071,77.81,15.83,0.70,1.43,0.26,2.21,2.16
4,California,38802500,06,CA,557,14.35,64500,23824254,2277229,282777,...,1753771,375682,1378089,61.40,5.87,0.73,14.30,0.41,13.66,4.52
5,Colorado,5355866,08,CO,86,16.06,63909,4607390,224809,49138,...,193331,30981,162350,86.03,4.20,0.92,3.04,0.17,3.92,3.61
6,Connecticut,3596677,09,CT,24,6.67,71346,2747943,380571,8893,...,116604,28025,88579,76.40,10.58,0.25,4.34,0.01,5.01,3.24
7,Delaware,935614,10,DE,9,9.62,61255,654904,203863,3121,...,28441,3500,24941,70.00,21.79,0.33,3.86,0.04,2.05,3.04
8,Florida,19893297,12,FL,327,16.44,49426,15357374,3280778,47032,...,501856,87649,414207,77.20,16.49,0.24,2.73,0.06,2.66,2.52
9,Georgia,10097343,13,GA,87,8.62,51244,6077438,3197929,28491,...,234124,36148,197976,60.19,31.67,0.28,3.85,0.04,2.81,2.32


In [33]:
# Plotly Info
py.tools.set_credentials_file(plotly_user, plotly_key)
init_notebook_mode(connected=True)

In [34]:
# Changing everything to strings
df_graph1 = df_states
for col in df_graph1.columns:
    df_graph1[col] = df_graph1[col].astype(str)

# Color Scale
scl = [[0.0, 'rgb(0,255,0)'],[0.2, 'rgb(100,255,0)'],[0.4, 'rgb(222,255,0)'],\
            [0.6, 'rgb(255,222,0)'],[0.8, 'rgb(255,100,0)'],[1.0, 'rgb(255,0,0)']]

# text overlay for states
df_graph1['text'] = "---" + df_graph1['STNAME'] + "---" + '<br>'\
            "Missing: " + df_graph1['Miss_Pop'] + '<br>'\
            "Population: " + df_graph1['POP'] + '<br>'\
            "Average Income: $" + df_graph1['Household Income'] + '<br>'\
            "---" + "Ethnic%" + "---" + '<br>'\
            "White: "+df_graph1['Whites%']+"% "+"Black: "+df_graph1['African Americans%']+"% "+":Native American: "+df_graph1['American Indians%']+"% "+"Asian: "+df_graph1['Asians%'] +"%" + '<br>'\
            "Pacific Islander: "+df_graph1['Pacific Islanders%']+"% "+"BiRacial: "+df_graph1['BiRacial%']+"% "+"Other: "+df_graph1['Others%']+"%"
            
'''
????
According to 2015 US Census Bureau estimates, California's population was 72.9% White, 
6.5% Black or African American, 14.7% Asian, 1.7% Native Americans, 0.5% Pacific Islander and 3.8% from two or more races. 
By ethnicity, 38.8% of the total population is Hispanic-Latino (of any race) and 61.4% Non-Hispanic (of any race). 
Hispanics are the largest ethnic group in California.
 - Wikipedia - 
'''

# Bunch of fancy settings for the plot
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_graph1['state_key'],
        z = df_graph1['Miss%'].astype(float),
        locationmode = 'USA-states',
        text = df_graph1['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Ratio Scale")
        ) ]

# Plot layout
layout = dict(
        title = '# of Missing People vs State Population<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig = dict( data=data, layout=layout )
iplot(fig, filename='0_MissingPeopleRatio')

### Trend By Decade

In [35]:
'''
<80s
90s
00s
10s>
plot on US MAP 
options:
1. dots by lat,lng https://plot.ly/python/scatter-plots-on-maps/
    a. slider https://plot.ly/python/sliders/
2. bubble grows bigger by decade https://plot.ly/python/bubble-maps/
'''

'\n<80s\n90s\n00s\n10s>\nplot on US MAP \noptions:\n1. dots by lat,lng https://plot.ly/python/scatter-plots-on-maps/\n    a. slider https://plot.ly/python/sliders/\n2. bubble grows bigger by decade https://plot.ly/python/bubble-maps/\n'

In [36]:
df_90 = df_NCMEC.loc[df_NCMEC['missing_year'] < 1990].groupby('missing_state').count()['name']
df_00 = df_NCMEC.loc[df_NCMEC['missing_year'] < 2000].groupby('missing_state').count()['name']
df_10 = df_NCMEC.loc[df_NCMEC['missing_year'] < 2010].groupby('missing_state').count()['name']
df_17 = df_NCMEC.groupby('missing_state').count()['name']

df_decades = pd.DataFrame(columns = ['1990','2000','2010','2017'],index = states_hash)

for i in range(len([df_90, df_00, df_10, df_17])):
    for state in list(df_decades.index):
        try:
            df_decades.set_value(state,'1990',df_90[state])
        except:
            pass
        try:
            df_decades.set_value(state,'2000',df_00[state])
        except:
            pass
        try:
            df_decades.set_value(state,'2010',df_10[state])
        except:
            pass
        try:
            df_decades.set_value(state,'2017',df_17[state])
        except:
            pass
df_decades = df_decades.fillna(0)

df_coord = pd.DataFrame(states_loc)
df_coord.sort_values(['state'], ascending = True)
df_decades['state'] = list(df_decades.index)
df_decades = df_decades.merge(df_coord,on='state')
df_decades.head()

,1990,2000,2010,2017,state,latitude,longitude
0,6,10,12,35,Alabama,32.7990,-86.8073
1,4,7,7,12,Alaska,61.3850,-152.2683
2,14,26,51,104,Arizona,33.7712,-111.3877
3,5,9,12,37,Arkansas,34.9513,-92.3809
4,83,132,213,557,California,36.1700,-119.7462


In [37]:
df_90 = df_decades[['1990','state','latitude','longitude']]
df_90 = df_90.rename(columns={'1990':'missing_count'})
df_00 = df_decades[['2000','state','latitude','longitude']]
df_00 = df_00.rename(columns={'2000':'missing_count'})
df_10 = df_decades[['2010','state','latitude','longitude']]
df_10 = df_10.rename(columns={'2010':'missing_count'})
df_17 = df_decades[['2017','state','latitude','longitude']]
df_17 = df_17.rename(columns={'2017':'missing_count'})
df_decades_long = pd.concat([df_90,df_00,df_10,df_17])
df_decades_long['text'] = ['{}<br>{} missing'.format(df_decades_long['state'].iloc[i],df_decades_long['missing_count'].iloc[i]) for i in range(len(df_decades_long))]
decade_years = ['1990','2000','2010','2017']

limits = [(0,50),(50,100),(100,150),(150,200)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)"]
decades = []

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_decades_long[lim[0]:lim[1]]
    decade = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['missing_count']*3,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{}'.format(decade_years[i]) )
    decades.append(decade)

layout = dict(
        title = 'MCMEC Database Growth By State Each Decade<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )

fig = dict( data=decades, layout=layout )
iplot( fig, validate=False, filename='1_DatabaseDecadeGrowth' )

### Top 3 Population vs Bottom 3 Population

In [38]:
df_Bot3Pop = df_state_pop.sort_values(["POP"], ascending = True).head(3)
df_Bot3Pop

NameError: name 'df_state_pop' is not defined

In [ ]:
df_Top3Pop = df_state_pop.sort_values(["POP"], ascending=False).head(3)
df_Top3Pop

In [ ]:
'''
state level data for top 3 and bot 3 states
gender, ethnicity, income, education

by county
crime, income, education

plot on state map seperated by county
dots (missing people) based on town
https://plot.ly/python/county-level-choropleth/
'''